In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
year ='2015'

In [9]:
data = pd.read_csv('/scratch/npatil4/networks/data/combined/data_'+year+'.csv',
                  sep=',',nrows=10)

In [12]:
del data['Unnamed: 0']

In [13]:
data.columns=['pick_time','drop_time',
              'dist_trip',
              'pick_long','pick_lat',
              'drop_long','drop_lat',
              'charge']

In [15]:
data.head()

,pick_time,drop_time,dist_trip,pick_long,pick_lat,drop_long,drop_lat,charge
0,2015-01-15 19:05:39,2015-01-15 19:23:42,1.59,-73.993896,40.750111,-73.974785,40.750618,17.05
1,2015-01-10 20:33:38,2015-01-10 20:53:28,3.30,-74.001648,40.724243,-73.994415,40.759109,17.80
2,2015-01-10 20:33:38,2015-01-10 20:43:41,1.80,-73.963341,40.802788,-73.951820,40.824413,10.80
3,2015-01-10 20:33:39,2015-01-10 20:35:31,0.50,-74.009087,40.713818,-74.004326,40.719986,4.80
4,2015-01-10 20:33:39,2015-01-10 20:52:58,3.00,-73.971176,40.762428,-74.004181,40.742653,16.30


In [10]:
sample = data

In [11]:
del data

In [12]:
sample.pick_time = pd.to_datetime(sample.pick_time)
sample.drop_time = pd.to_datetime(sample.drop_time)
sample.dtypes

pick_time    datetime64[ns]
drop_time    datetime64[ns]
dist_trip           float64
pick_long           float64
pick_lat            float64
drop_long           float64
drop_lat            float64
charge              float64
dtype: object

In [13]:
import warnings
warnings.simplefilter("ignore")

sample['pick_long'] = np.round(sample['pick_long'],3)
sample['pick_lat'] = np.round(sample['pick_lat'],3)
sample['drop_long'] = np.round(sample['drop_long'],3)
sample['drop_lat'] = np.round(sample['drop_lat'],3)

sample['year'] = 0
sample['month'] = 0
sample['day'] = 0
sample['hour'] = 0
sample['duration'] = 0

In [14]:
sample.dtypes

pick_time    datetime64[ns]
drop_time    datetime64[ns]
dist_trip           float64
pick_long           float64
pick_lat            float64
drop_long           float64
drop_lat            float64
charge              float64
year                  int64
month                 int64
day                   int64
hour                  int64
duration              int64
dtype: object

In [15]:
print 'converting time'
sample['year'] = sample.pick_time.dt.year
sample['month'] = sample.pick_time.dt.month
sample['day'] = sample.pick_time.dt.day
sample['hour'] = sample.pick_time.dt.hour
sample['weekDay'] = sample.pick_time.dt.weekday
sample['weekNo'] = sample.pick_time.dt.week

print 'getting duration'
sample['duration'] = (sample.drop_time-sample.pick_time)/ np.timedelta64(1, 's')
sample['trips']=1


del sample['pick_time']
del sample['drop_time']

converting time
getting duration


mask = (sample.pick_lat>40) & (sample.drop_lat<42)  & (sample.pick_long>-75)& (sample.drop_long<-73)
sample = sample[mask]

In [16]:
print 'start'
sample['start'] = '('+sample['pick_lat'].astype('str')+','+sample['pick_long'].astype('str')+')'
print 'stop'
sample['stop'] = '('+sample['drop_lat'].astype('str')+','+sample['drop_long'].astype('str')+')'

start
stop


In [18]:
sample.head()

,dist_trip,pick_long,pick_lat,drop_long,drop_lat,charge,year,month,day,hour,duration,weekDay,weekNo,trips,start,stop
0,1.59,-73.994,40.750,-73.975,40.751,17.05,2015,1,15,19,1083.0,3,3,1,"(40.75,-73.994)","(40.751,-73.975)"
1,3.30,-74.002,40.724,-73.994,40.759,17.80,2015,1,10,20,1190.0,5,2,1,"(40.724,-74.002)","(40.759,-73.994)"
2,1.80,-73.963,40.803,-73.952,40.824,10.80,2015,1,10,20,603.0,5,2,1,"(40.803,-73.963)","(40.824,-73.952)"
3,0.50,-74.009,40.714,-74.004,40.720,4.80,2015,1,10,20,112.0,5,2,1,"(40.714,-74.009)","(40.72,-74.004)"
4,3.00,-73.971,40.762,-74.004,40.743,16.30,2015,1,10,20,1159.0,5,2,1,"(40.762,-73.971)","(40.743,-74.004)"


In [19]:
sample.to_csv('/public/npatil4/networks/data/combined/data_'+year+'_mod2.txt',
                         sep='|',header=True,index=False)

#### Total trips from one node to another : in one year

In [29]:
data_grouped_year = sample.groupby(['start','stop',
                                    'pick_lat','pick_long',
                                    'drop_lat','drop_long'],
                                   as_index=False).agg({'trips':sum,
                                                        'dist_trip':sum,
                                                        'charge':sum,
                                                        'duration':sum})

data_grouped_year.to_csv('/public/npatil4/networks/data/combined/data_'+year+'_withNodes_year.txt',
                         sep='|',header=True,index=False)

del data_grouped_year

#### Total trips from one node to another : in one month

In [30]:
data_grouped_month = sample.groupby(['start','stop',
                                  'pick_lat','pick_long',
                                  'drop_lat','drop_long',
                                  'month'],
                                  as_index=False).agg({'trips':sum,
                                                       'dist_trip':sum,
                                                       'charge':sum,
                                                       'duration':sum})


data_grouped_month.to_csv('/public/npatil4/networks/data/combined/data_'+year+'_withNodes_month.txt',
                         sep='|',header=True,index=False)
del data_grouped_month

#### Total trips from one node to another : in one day

In [31]:
data_grouped_day = sample.groupby(['start','stop',
                                  'pick_lat','pick_long',
                                  'drop_lat','drop_long',
                                  'month','day'],
                                   as_index=False).agg({'trips':sum,
                                                        'dist_trip':sum,
                                                        'charge':sum,
                                                        'duration':sum})
data_grouped_day = pd.DataFrame(data_grouped_day)


data_grouped_day.to_csv('/public/npatil4/networks/data/combined/data_'+year+'_withNodes_day.txt',
                         sep='|',header=True,index=False)
del data_grouped_day

#### Total trips from one node to another : in one hour

In [32]:
data_grouped_hour = sample.groupby(['start','stop',
                                  'pick_lat','pick_long',
                                  'drop_lat','drop_long',
                                  'month','day',
                                  'hour'],
                                  as_index=False).agg({'trips':sum,
                                                       'dist_trip':sum,
                                                       'charge':sum,
                                                       'duration':sum})
data_grouped_hour = pd.DataFrame(data_grouped_hour)


data_grouped_hour.to_csv('/public/npatil4/networks/data/combined/data_'+year+'_withNodes_hour.txt',
                         sep='|',header=True,index=False)
del data_grouped_hour